## Artificial Intelligence Assignment 3
## No3. CNN7 Find Best CNN
## 20132651 Sungjae Lee

#### 위에서 배운 내용을 바탕으로, 최대한의 Accuracy 를 내보고자 저만의 CNN을 구현해 보았습니다. 
#### Deep Kernal 을 사용하는 동시에 다양한 규제 기법 및 데이터 확대를 하였습니다.
#### 학습량이 적은 것이 문제라 생각하여 epoch 를 10으로 진행하였으나, 시간이 너무 오래걸려 2까지 하였습니다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform_basic = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_hflip = transforms.Compose([
    transforms.functional.hflip,
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_color = transforms.Compose([
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:

trainset_basic = torchvision.datasets.CIFAR10(root = './data', train = True,
                                             download = True, transform = transform_basic)

trainset_hflip = torchvision.datasets.CIFAR10(root = './data', train = True,
                                             download = True, transform = transform_hflip)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_basic)

trainset_all = torch.utils.data.ConcatDataset([trainset_basic, trainset_hflip])


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
trainloader = torch.utils.data.DataLoader(trainset_all,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)

In [5]:
import torch.nn as nn
import torch.nn.functiondal as F

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)        
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.conv4 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-03, betas = (0.9, 0.999), weight_decay = 0.001)

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [8]:
for epoch in range(10):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 5000 == 4999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

[1,  5000] loss : 8.710
[1, 10000] loss : 7.158
[1, 15000] loss : 6.464
[1, 20000] loss : 6.044
[1, 25000] loss : 5.838
[2,  5000] loss : 5.634
[2, 10000] loss : 5.546
[2, 15000] loss : 5.421
[2, 20000] loss : 5.353
[2, 25000] loss : 5.326


Process Process-6:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeou

KeyboardInterrupt: 

### (6) 화면 출력 확인 및 일반화 성능 서술

In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

## 확실히 성능이 약간 더 좋아진 모습을 확인할 수 있었습니다.
## 더 빠른 연산으로 epoch 를 10까지 진행하면 또 다른 결과가 있을 것으로 보입니다.

Accuracy of the network on the 10000 test images :62 %


In [10]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [11]:
for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 64 %
Accuracy of   car: 72 %
Accuracy of  bird: 46 %
Accuracy of   cat: 32 %
Accuracy of  deer: 78 %
Accuracy of   dog: 49 %
Accuracy of  frog: 66 %
Accuracy of horse: 65 %
Accuracy of  ship: 79 %
Accuracy of truck: 72 %
